# Video Playback Detection

Source Code is adapted from [here](https://learnopencv.com/read-write-and-display-a-video-using-opencv-cpp-python/)

# Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Libraries

In [3]:

import numpy as np
import cv2
from google.colab.patches import cv2_imshow    # cv.imshow() is known to crash Google Colab notebooks. This is the alternative suggested by Colab.
import os
from tensorflow.keras.models import load_model

# Load Age and Gender models

In [4]:
# Getting the trained CNN model from the source link.
#!wget -qq "https://drive.google.com/drive/folders/1-2HxPUcCTdV0InQeJG3rFWOxp_xF7sm0?usp=sharing" -O "final_cnn_model_checkpoint.h5"

# Loading the trained CNN model for age classification, and defining a list of age-ranges as defined in the model.
age_model = load_model("/content/drive/MyDrive/Colab Notebooks/models/age/final_age_model_checkpoint_7403.h5")
age_ranges = ['0-1','2-4','5-12','13-19','20-39','50-69','60-116']

gender_model = load_model("/content/drive/MyDrive/Age_Classification_with_Faces/input_output/gender_cnn_1_checkpoint.h5")
gender_ranges = ['MALE','FEMALE']

# Load Face Detection Model 

we will be using the Haarcascade_frontalface model

In [5]:


# Getting the trained CNN model from the source link.
#!wget -qq "https://drive.google.com/uc?export=download&id=1Gcz4wc8iA1SHfV9REcK4i74Tf9vaETq7" -O "haarcascade_frontalface_default.xml"

# Importing the Haar Cascades classifier XML file.
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to detect face and use model to predict

In [44]:

# Defining a function to shrink the detected face region by a scale for better prediction in the model.

def shrink_face_roi(x, y, w, h, scale=0.9):
    wh_multiplier = (1-scale)/2
    x_new = int(x + (w * wh_multiplier))
    y_new = int(y + (h * wh_multiplier))
    w_new = int(w * scale)
    h_new = int(h * scale)
    return (x_new, y_new, w_new, h_new)


# Defining a function to create the predicted age overlay on the image by centering the text.

def create_age_text(img, age, pct_age, gender, pct_gender, x, y, w, h):

    # Defining font, scales and thickness.
    fontFace = cv2.FONT_HERSHEY_SIMPLEX
    age_scale = 1.0
    pct_age_scale = 0.50
    gender_scale = 1.0
    pct_gender_scale = 0.50

    # Getting width, height and baseline of age text and "years old".
    (age_width, age_height), age_bsln = cv2.getTextSize(age, fontFace=fontFace, fontScale=age_scale, thickness=2)
    (pct_age_width, pct_age_height), pct_age_bsln = cv2.getTextSize(pct_age, fontFace=fontFace, fontScale=pct_age_scale, thickness=1)
    (gender_width, gender_height), gender_bsln = cv2.getTextSize(gender, fontFace=fontFace, fontScale=gender_scale, thickness=2)
    (pct_gender_width, pct_gender_height), pct_gender_bsln = cv2.getTextSize(pct_gender, fontFace=fontFace, fontScale=pct_gender_scale, thickness=1)


    # Calculating center point coordinates of text background rectangle.
    x_center = x + (w/2)
    y_age_center = y + h + 20
    y_pct_age_center = y + h + 48
    y_gender_center = y + h + 75
    y_pct_gender_center = y + h + 103


    # Calculating bottom left corner coordinates of text based on text size and center point of background rectangle calculated above.
    x_age_org = int(round(x_center - (age_width / 2)))
    y_age_org = int(round(y_age_center + (age_height / 2)))
    x_pct_age_org = int(round(x_center - (pct_age_width / 2)))
    y_pct_age_org = int(round(y_pct_age_center + (pct_age_height / 2)))
    x_gender_org = int(round(x_center - (gender_width / 2)))
    y_gender_org = int(round(y_gender_center + (gender_height / 2)))
    x_pct_gender_org = int(round(x_center - (pct_age_width / 2)))
    y_pct_gender_org = int(round(y_pct_gender_center + (pct_gender_height / 2)))

    face_age_background = cv2.rectangle(img, (x-1, y+h), (x+w+1, y+h+94), (200, 0, 0), cv2.FILLED)
    face_age_text = cv2.putText(img, age, org=(x_age_org, y_age_org), fontFace=fontFace, fontScale=age_scale, thickness=2, color=(255, 255, 255), lineType=cv2.LINE_AA)
    pct_age_text = cv2.putText(img, pct_age, org=(x_pct_age_org, y_pct_age_org), fontFace=fontFace, fontScale=pct_age_scale, thickness=1, color=(255, 255, 255), lineType=cv2.LINE_AA)
    face_gender_text = cv2.putText(img, gender, org=(x_gender_org, y_gender_org), fontFace=fontFace, fontScale=gender_scale, thickness=2, color=(255, 255, 255), lineType=cv2.LINE_AA)
    pct_gender_text = cv2.putText(img, pct_gender, org=(x_pct_gender_org, y_pct_gender_org), fontFace=fontFace, fontScale=pct_gender_scale, thickness=1, color=(255, 255, 255), lineType=cv2.LINE_AA)

    return (face_age_background, face_age_text, face_gender_text)


# Defining a function to find faces in an image and then classify each found face into age-ranges defined above.

def classify_age(img):

    # Making a copy of the image for overlay of ages and making a grayscale copy for passing to the loaded model for age classification.
    img_copy = np.copy(img)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detecting faces in the image using the face_cascade loaded above and storing their coordinates into a list.
    faces = face_cascade.detectMultiScale(img_copy, scaleFactor=1.2, minNeighbors=6, minSize=(100, 100))
    # print(f"{len(faces)} faces found.")

    # Looping through each face found in the image.
    for i, (x, y, w, h) in enumerate(faces):

        # Drawing a rectangle around the found face.
        face_rect = cv2.rectangle(img_copy, (x, y), (x+w, y+h), (0, 100, 0), thickness=2)
        
        # Predicting the age of the found face using the model loaded above.
        x2, y2, w2, h2 = shrink_face_roi(x, y, w, h)
        face_roi_1 = img_copy[y2:y2+h2, x2:x2+w2]
        face_roi_1 = cv2.resize(face_roi_1, (224, 224))
        face_roi_1 = face_roi_1.reshape(-1, 224, 224, 3)
        face_roi_2 = img_gray[y2:y2+h2, x2:x2+w2]
        face_roi_2 = cv2.resize(face_roi_2, (200, 200))
        face_roi_2 = face_roi_2.reshape(-1, 200, 200, 1)
        face_age = age_ranges[np.argmax(age_model.predict(face_roi_1))]
        face_age_pct = f"({round(np.max(age_model.predict(face_roi_1))*100, 2)}%)"
        face_gender = gender_ranges[np.argmax(gender_model.predict(face_roi_2))]
        face_gender_pct = f"({round(np.max(gender_model.predict(face_roi_2))*100, 2)}%)"
        
        
        # Calling the above defined function to create the predicted age overlay on the image.
        face_age_background, face_age_text, face_gender_text = create_age_text(img_copy, face_age, face_age_pct, face_gender, face_gender_pct, x, y, w, h)
        # print(f"Age prediction for face {i+1} : {face_age} years old")

    return img_copy


# Defining a function to return the image filepath with a new filename.
# If INPUT filepath is "my_folder1/my_folder2/my_image.jpg", OUTPUT filepath will be "my_folder1/my_folder2/my_image_WITH_AGE.jpg"

def new_img_name(org_img_path):
    img_path, img_name_ext = os.path.split(org_img_path)
    img_name, img_ext = os.path.splitext(img_name_ext)

    new_img_name_ext = img_name+"_WITH_AGE"+img_ext
    new_img_path = os.path.join(img_path, new_img_name_ext)

    return new_img_path


# Defining a function to return the video filepath with a new filename.
# If INPUT filepath is "my_folder1/my_folder2/my_video.mp4", OUTPUT filepath will be "my_folder1/my_folder2/my_video_WITH_AGE.mp4"

def new_vid_name(org_vid_path):
    vid_path, vid_name_ext = os.path.split(org_vid_path)
    vid_name, vid_ext = os.path.splitext(vid_name_ext)

    new_vid_name_ext = vid_name+"_WITH_AGE_latest"+".mp4"
    new_vid_path = os.path.join(vid_path, new_vid_name_ext)

    return new_vid_path

# Upload Video

In [45]:
# Define Video File
video = '/content/drive/MyDrive/videos/shua_video.mov'

In [46]:
# Reading the video from filepath provided above and passing it through the age clasification method defined above.
# Source 1: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
# Source 2: https://www.learnopencv.com/read-write-and-display-a-video-using-opencv-cpp-python/

# Creating a VideoCapture object.
cap = cv2.VideoCapture(video)

# Getting the video frame width and height.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Defining the codec and creating a VideoWriter object to save the output video at the same location.
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
new_my_video = new_vid_name(video)
out = cv2.VideoWriter(new_my_video, fourcc, 18, (frame_width, frame_height))

while(cap.isOpened()):
    
    # Grabbing each individual frame, frame-by-frame.
    ret, frame = cap.read()
    
    if ret==True:
        
        # Running age detection on the grabbed frame.
        age_img = classify_age(frame)
        
        # Saving frame to output video using the VideoWriter object defined above.
        out.write(age_img)

    else:
        break

# Releasing the VideoCapture and VideoWriter objects.
cap.release()
out.release()
print(f"Saved to {new_my_video}")

Saved to /content/drive/MyDrive/videos/shua_video_WITH_AGE_latest.mp4
